In [49]:
!pip install shutup
import shutup
shutup.please()

### Test random enviroment

In [7]:
import gym
import random

In [ ]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [55]:
episodes = 15
for episode in range(1, episodes + 1):
    state = env.reset()
    score = 0
    done = False

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward

    print(f"Episode: {episode}, reward: {score}")

Episode: 1, reward: 31.0
Episode: 2, reward: 12.0
Episode: 3, reward: 19.0
Episode: 4, reward: 24.0
Episode: 5, reward: 13.0
Episode: 6, reward: 13.0
Episode: 7, reward: 32.0
Episode: 8, reward: 11.0
Episode: 9, reward: 35.0
Episode: 10, reward: 14.0
Episode: 11, reward: 10.0
Episode: 12, reward: 20.0
Episode: 13, reward: 32.0
Episode: 14, reward: 42.0
Episode: 15, reward: 15.0


### Create a deep learning model with Keras

In [44]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers.legacy import Adam

In [ ]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))

    return model

In [40]:
model = build_model(states, actions)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 4)                 0         
                                                                 
 dense_9 (Dense)             (None, 24)                120       
                                                                 
 dense_10 (Dense)            (None, 24)                600       
                                                                 
 dense_11 (Dense)            (None, 2)                 50        
                                                                 
Total params: 770 (3.01 KB)
Trainable params: 770 (3.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Build agent with Keras-RL

In [25]:
import tensorflow
from keras import __version__
tensorflow.keras.__version__ = __version__

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import  SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    agent = DQNAgent(policy=policy, model=model, memory=memory,
                     nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return agent

In [50]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 97s 10ms/step - reward: 1.0000
50 episodes - episode_reward: 198.180 [152.000, 200.000] - loss: 10.235 - mae: 43.137 - mean_q: 86.769

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 100s 10ms/step - reward: 1.0000
54 episodes - episode_reward: 184.685 [82.000, 200.000] - loss: 13.234 - mae: 42.871 - mean_q: 86.020

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 111s 11ms/step - reward: 1.0000
51 episodes - episode_reward: 196.000 [47.000, 200.000] - loss: 10.103 - mae: 41.393 - mean_q: 83.237

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: 1.0000
51 episodes - episode_reward: 197.725 [102.000, 200.000] - loss: 10.500 - mae: 42.831 - mean_q: 85.955

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 116s 12ms/step - reward: 

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)

In [53]:
visual = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


### Save

In [56]:
dqn.save_weights("dqn_weights.h5f", overwrite=True)